# TWSE股票爬蟲
### 作者：許喬雅

<br>

### 載入套件

In [9]:
from distutils.util import strtobool
import requests
from bs4 import BeautifulSoup as Soup
import json
import pandas as pd
import time
from tqdm import tqdm
import time
import re
import random
from datetime import tzinfo, timedelta, datetime
from dateutil.relativedelta import relativedelta
import os
import sys

### 設定工作目錄

In [10]:
os.chdir("D:\新增資料夾\大四上\intern\股票爬蟲")

### 輸入股票代號、搜尋起始年月、是否分頁

In [8]:
stocknum = input("請輸入股票代碼")
start_year = input("請輸入起始年(西元)：")
start_mon = input("請輸入起始月(01~12)：")
end_year = input("請輸入結束年(西元)：")
end_mon = input("請輸入結束月(01~12)：")
by_year = input('是否照年份分別輸出 (True/False)：')

# 將 by_year 轉換成boolean值
by_year = eval(by_year)

請輸入股票代碼0050
請輸入起始年(西元)：2016
請輸入起始月(01~12)：08
請輸入結束年(西元)：2020
請輸入結束月(01~12)：06
是否照年份分別輸出 (True/False)：True


### 偵錯起始年月大於結束年月

In [11]:
if(int(start_year) == int(end_year)):
    if(int(start_month) > int(end_month)):
        print('錯誤月份，起始年月大於結束年月')
        sys.exit()

### 偵錯起始年月大於結束年月

In [12]:
end_date = end_year + end_mon
end_datetime = datetime.strptime(end_date,'%Y%m')
now = datetime.today()

if(end_datetime > now):
    print('錯誤月份，結束年月大於現在時間')
    sys.exit()


### 偵錯輸入未來時間

In [13]:
end_date = end_year + end_mon
end_datetime = datetime.strptime(end_date,'%Y%m')
now = datetime.today()

if(end_datetime > now):
    print('錯誤月份，結束年月大於現在時間')
    sys.exit()


### 起始、結束年月資料處理

In [14]:
# ------起始年與起始月字串合併，ex：'2016'+'08'='201608'------
start_date = start_year + start_mon

# ------結束年與結束月字串合併，ex：'2020'+'06'='202006'------
end_date = end_year + end_mon

# ------先把end_date儲存起來，後面excelwriter會用到------
end_date_real = end_date


In [10]:
start_date

'201608'

In [11]:
end_date

'202006'

### end_date往後調一個月，之後while迴圈才會正常判斷
1. 若end_date的月份為12，要用datetime調整格式到明年1月
2. 若end_date的月份不為12，字串直接轉換為整數加1，再轉為字串

In [15]:
if(end_date[-2:] == '12'):
    end_date = datetime.strptime(end_date,'%Y%m')
    end_date = end_date + relativedelta(months=1)
    end_date = datetime.strftime(end_date , '%Y%m')
else:
    end_date = str(int(end_date) +1)


In [13]:
end_date


'202007'

### 建立excelwriter
1. 若by_year為True，會建立名稱為股票代號的資料夾，並把檔案除儲存在裡面
2. 若by_year為False，直接建立名稱為股票代號的excel檔

In [14]:
if(by_year == True):
    os.mkdir('股票代號_'+ stocknum)
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter('股票代號_'+ stocknum+'/'+start_year+'年.xlsx', engine='xlsxwriter')
else:
    writer = pd.ExcelWriter('股票代號_'+ stocknum+'.xlsx', engine='xlsxwriter')


### 進入網站抓取資料

In [16]:
url = 'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=' + start_date + '01&stockNo=' + stocknum +'&_=1598235282210'

# ------用time.sleep防止IP被鎖------
time.sleep(2)
req = requests.get(url)
soup = Soup(req.text)
js = json.loads(soup.text)
print(start_date,'checks')


201608 checks


In [16]:
js 


{'stat': 'OK',
 'date': '20160801',
 'title': '105年08月 0050 元大台灣50       各日成交資訊',
 'fields': ['日期', '成交股數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數'],
 'data': [['105/08/01',
   '10,407,371',
   '720,963,537',
   '68.80',
   '69.45',
   '68.80',
   '69.30',
   '+0.75',
   '1,682'],
  ['105/08/02',
   '7,116,974',
   '492,812,674',
   '69.25',
   '69.40',
   '69.10',
   '69.30',
   ' 0.00',
   '1,216'],
  ['105/08/03',
   '13,801,107',
   '946,584,522',
   '68.50',
   '68.90',
   '68.05',
   '68.75',
   '-0.55',
   '1,470'],
  ['105/08/04',
   '10,674,699',
   '733,253,890',
   '68.85',
   '68.95',
   '68.35',
   '68.90',
   '+0.15',
   '1,144'],
  ['105/08/05',
   '12,422,864',
   '864,171,627',
   '69.10',
   '69.75',
   '68.90',
   '69.60',
   '+0.70',
   '2,242'],
  ['105/08/08',
   '7,902,947',
   '553,043,688',
   '70.00',
   '70.25',
   '69.70',
   '70.05',
   '+0.45',
   '1,664'],
  ['105/08/09',
   '11,568,485',
   '812,755,084',
   '70.00',
   '70.45',
   '69.95',
  

### 所需資料在data裡面

In [17]:
fin_data = js['data']
fin_data = pd.DataFrame(fin_data)
fin_data

,0,1,2,3,4,5,6,7,8
0,105/08/01,"10,407,371","720,963,537",68.80,69.45,68.80,69.30,+0.75,"1,682"
1,105/08/02,"7,116,974","492,812,674",69.25,69.40,69.10,69.30,0.00,"1,216"
2,105/08/03,"13,801,107","946,584,522",68.50,68.90,68.05,68.75,-0.55,"1,470"
3,105/08/04,"10,674,699","733,253,890",68.85,68.95,68.35,68.90,+0.15,"1,144"
4,105/08/05,"12,422,864","864,171,627",69.10,69.75,68.90,69.60,+0.70,"2,242"
5,105/08/08,"7,902,947","553,043,688",70.00,70.25,69.70,70.05,+0.45,"1,664"
6,105/08/09,"11,568,485","812,755,084",70.00,70.45,69.95,70.40,+0.35,"2,128"
7,105/08/10,"6,497,798","457,603,790",70.30,70.60,70.05,70.60,+0.20,"1,312"
8,105/08/11,"10,903,420","764,847,436",70.20,70.55,69.45,69.95,-0.65,"1,445"
9,105/08/12,"5,258,706","369,627,870",70.15,70.50,70.05,70.20,+0.25,"1,246"


In [19]:
js['fields']

['日期', '成交股數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數']

### 設定欄位名稱為fields

In [18]:
fin_data.columns = js['fields']
fin_data

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,105/08/01,"10,407,371","720,963,537",68.80,69.45,68.80,69.30,+0.75,"1,682"
1,105/08/02,"7,116,974","492,812,674",69.25,69.40,69.10,69.30,0.00,"1,216"
2,105/08/03,"13,801,107","946,584,522",68.50,68.90,68.05,68.75,-0.55,"1,470"
3,105/08/04,"10,674,699","733,253,890",68.85,68.95,68.35,68.90,+0.15,"1,144"
4,105/08/05,"12,422,864","864,171,627",69.10,69.75,68.90,69.60,+0.70,"2,242"
5,105/08/08,"7,902,947","553,043,688",70.00,70.25,69.70,70.05,+0.45,"1,664"
6,105/08/09,"11,568,485","812,755,084",70.00,70.45,69.95,70.40,+0.35,"2,128"
7,105/08/10,"6,497,798","457,603,790",70.30,70.60,70.05,70.60,+0.20,"1,312"
8,105/08/11,"10,903,420","764,847,436",70.20,70.55,69.45,69.95,-0.65,"1,445"
9,105/08/12,"5,258,706","369,627,870",70.15,70.50,70.05,70.20,+0.25,"1,246"


### 將index設定為stocknum

In [19]:
fin_data['股票代號'] = stocknum
fin_data = fin_data.set_index('股票代號')
fin_data

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
股票代號,,,,,,,,,
0050,105/08/01,"10,407,371","720,963,537",68.80,69.45,68.80,69.30,+0.75,"1,682"
0050,105/08/02,"7,116,974","492,812,674",69.25,69.40,69.10,69.30,0.00,"1,216"
0050,105/08/03,"13,801,107","946,584,522",68.50,68.90,68.05,68.75,-0.55,"1,470"
0050,105/08/04,"10,674,699","733,253,890",68.85,68.95,68.35,68.90,+0.15,"1,144"
0050,105/08/05,"12,422,864","864,171,627",69.10,69.75,68.90,69.60,+0.70,"2,242"
0050,105/08/08,"7,902,947","553,043,688",70.00,70.25,69.70,70.05,+0.45,"1,664"
0050,105/08/09,"11,568,485","812,755,084",70.00,70.45,69.95,70.40,+0.35,"2,128"
0050,105/08/10,"6,497,798","457,603,790",70.30,70.60,70.05,70.60,+0.20,"1,312"
0050,105/08/11,"10,903,420","764,847,436",70.20,70.55,69.45,69.95,-0.65,"1,445"


### 將時間往後推移到下一個月

In [26]:
# ------將字串轉換成datetime形式------
start_datetime = datetime.strptime(start_date,'%Y%m')

# ------將start_datetime加上一個月------
start_datetime = start_datetime + relativedelta(months=1)

# ------將start_datetime轉換回字串形式------
start_date = datetime.strftime(start_datetime , '%Y%m')


In [27]:
start_date


'201609'

### excelwriter儲存
1. 若目前儲存月份為12月，且by_year為True，且不為搜尋的最後一個月份 -> 儲存此份excelwriter，並建立下一年的excelwriter
2. 若目前儲存月份為12月，且by_year為True，且為搜尋的最後一個月份(end_date_real) -> 儲存此分頁
3. 若目前儲存月份不為12月，或by_year為False -> 儲存此分頁

In [28]:
if (start_date[-2:] == '12' and by_year == True and start_date != end_date_real):
    fin_data.to_excel(writer, sheet_name = start_date)
    print(start_date,'saves!')
    writer.save()
    writer = pd.ExcelWriter('股票代號_'+ stocknum+'/'+str(int(start_date[:-2])+1)+'年.xlsx', engine='xlsxwriter')
elif(start_date[-2:] == '12' and by_year == True and start_date == end_date_real):
    fin_data.to_excel(writer, sheet_name = start_date)
    print(start_date,'saves!')
else:
    fin_data.to_excel(writer, sheet_name = start_date)
    print(start_date,'saves!')


201609 saves!


In [32]:
def datetime_plus(start_date):
    # ------將字串轉換成datetime形式------
    start_datetime = datetime.strptime(start_date,'%Y%m')
    # ------將start_datetime加上一個月------
    start_datetime = start_datetime + relativedelta(months=1)
    # ------將start_datetime轉換回字串形式------
    start_date = datetime.strftime(start_datetime , '%Y%m')
    return start_date


### 爬取所有月份的資料，若start_date不等於結束年月，則繼續爬取

In [43]:
while start_date != end_date :
    # ------進入網站抓取資料------
    url = 'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=' + start_date + '01&stockNo=' + stocknum +'&_=1598235282210'
   
    # ------用time.sleep防止IP被鎖------
    time.sleep(2.5)
    req = requests.get(url)
    soup = Soup(req.text)
    js = json.loads(soup.text)
    print(start_date,'checks')
    
    # ------所需資料在data裡面------
    fin_data = js['data']
    fin_data = pd.DataFrame(fin_data)
    
    # ------設定欄位名稱為fields------
    fin_data.columns = js['fields']
    
    # ------將index設定為stocknum------
    fin_data['股票代號'] = stocknum
    fin_data = fin_data.set_index('股票代號')
    
    # ------excelwriter儲存------
    '''
    1. 若目前儲存月份為12月，且by_year為True，且不為搜尋的最後一個月份 -> 儲存此份excelwriter，並建立下一年的excelwriter
    2. 若目前儲存月份為12月，且by_year為True，且為搜尋的最後一個月份(end_date_real) -> 儲存此分頁
    3. 若目前儲存月份不為12月，或by_year為False -> 儲存此分頁
    '''
    if (start_date[-2:] == '12' and by_year == True and start_date != end_date_real):
        fin_data.to_excel(writer, sheet_name = start_date)
        print(start_date,'saves!')
        writer.save()
        writer = pd.ExcelWriter('股票代號_'+ stocknum+'/'+str(int(start_date[:-2])+1)+'年.xlsx', engine='xlsxwriter')
    elif(start_date[-2:] == '12' and by_year == True and start_date == end_date_real):
        fin_data.to_excel(writer, sheet_name = start_date)
        print(start_date,'saves!')
    else:
        fin_data.to_excel(writer, sheet_name = start_date)
        print(start_date,'saves!')
        
    # ------將時間往後推移到下一個月------
    start_date = datetime_plus(start_date)
    
    
# ------while迴圈跑完，執行excelwriter儲存------
writer.save()


201608 checks
201608 saves!
201609 checks
201609 saves!
201610 checks
201610 saves!
201611 checks
201611 saves!
201612 checks
201612 saves!
201701 checks
201701 saves!
201702 checks
201702 saves!
201703 checks
201703 saves!
201704 checks
201704 saves!
201705 checks
201705 saves!
201706 checks
201706 saves!
201707 checks
201707 saves!
201708 checks
201708 saves!
201709 checks
201709 saves!
201710 checks
201710 saves!
201711 checks
201711 saves!
201712 checks
201712 saves!
201801 checks
201801 saves!
201802 checks
201802 saves!
201803 checks
201803 saves!
201804 checks
201804 saves!
201805 checks
201805 saves!
201806 checks
201806 saves!
201807 checks
201807 saves!
201808 checks
201808 saves!
201809 checks
201809 saves!
201810 checks
201810 saves!
201811 checks
201811 saves!
201812 checks
201812 saves!
201901 checks
201901 saves!
201902 checks
201902 saves!
201903 checks
201903 saves!
201904 checks
201904 saves!
201905 checks
201905 saves!
201906 checks
201906 saves!
201907 checks
201907